## Librerias

In [1]:
import pandas as pd
import numpy as np
import re
import openpyxl
import unicodedata

## Carga de archivos

In [2]:
# Carga de los archivos base de datos maestra y Scopus
maestra_path = 'C:/Users/Rodrigo/Desktop/Roxana Files/Scopus/maestra.xlsx'
scopus_path = 'C:/Users/Rodrigo/Desktop/Roxana Files/Scopus/scopus_resultados_beta.xlsx'

# Cargar la base de datos maestra
df_maestra = pd.read_excel(maestra_path, sheet_name='BaseMaestra')

# Cargar la base de datos de Scopus
df_scopus = pd.read_excel(scopus_path)

print(f"Filas en la base de datos maestra: {len(df_maestra)}")
print(f"Filas en la base de datos Scopus: {len(df_scopus)}")

Filas en la base de datos maestra: 3166
Filas en la base de datos Scopus: 23179


In [4]:
import unicodedata

# Función para normalizar nombres (eliminar tildes y convertir a mayúsculas)
def normalizar_nombre(nombre):
    # Verificar si el valor es una cadena
    if isinstance(nombre, str):
        # Convertir a mayúsculas, eliminar tildes y caracteres especiales
        nombre = nombre.upper()
        nombre = ''.join(c for c in unicodedata.normalize('NFKD', nombre) if unicodedata.category(c) != 'Mn')
        nombre = nombre.replace("-", " ")  # Reemplazar guiones por espacios
        nombre = nombre.strip()
    else:
        nombre = ''  # Manejar valores no válidos como cadenas vacías
    return nombre

# Aplicar la normalización en la base maestra
df_maestra['Primer_Nombre'] = df_maestra['Primer_Nombre'].apply(normalizar_nombre)
df_maestra['Segundo_Nombre'] = df_maestra['Segundo_Nombre'].apply(normalizar_nombre)
df_maestra['Apellido_Paterno'] = df_maestra['Apellido_Paterno'].apply(normalizar_nombre)
df_maestra['Apellido_Materno'] = df_maestra['Apellido_Materno'].apply(normalizar_nombre)

# Aplicar la normalización en los nombres de Scopus
df_scopus['nombre_completo_scopus'] = df_scopus['nombre_completo_scopus'].apply(normalizar_nombre)

# Mostrar ejemplos de nombres normalizados
print("Ejemplos de nombres normalizados en la base maestra:")
print(df_maestra[['Primer_Nombre', 'Segundo_Nombre', 'Apellido_Paterno', 'Apellido_Materno']].head())

print("Ejemplos de nombres normalizados en Scopus:")
print(df_scopus['nombre_completo_scopus'].head())


Ejemplos de nombres normalizados en la base maestra:
  Primer_Nombre Segundo_Nombre Apellido_Paterno Apellido_Materno
0       ABELINO        ENRIQUE          JIMENEZ          GAJARDO
1       ABRAHAM       BENJAMIN            NOVOA            LAGOS
2         ACCEL        NICOLAS           ABARCA           PROUZA
3      ADELAIDA         BARROS           BARROS                 
4        ADOLFO        ANTONIO            ROCCO            ROCCO
Ejemplos de nombres normalizados en Scopus:
0                  ABELINO JIMENEZ
1           ABELINO VARGAS JIMENEZ
2                                 
3                    ENRIQUE PEREZ
4    PEDRO ENRIQUE SANCHEZ JIMENEZ
Name: nombre_completo_scopus, dtype: object


In [5]:
# Crear combinaciones de nombres en la base maestra
df_maestra = df_maestra.assign(
    comb_1=df_maestra['Primer_Nombre'] + " " + df_maestra['Segundo_Nombre'] + " " + df_maestra['Apellido_Paterno'] + " " + df_maestra['Apellido_Materno'],
    comb_2=df_maestra['Primer_Nombre'] + " " + df_maestra['Segundo_Nombre'] + " " + df_maestra['Apellido_Materno'] + " " + df_maestra['Apellido_Paterno'],
    comb_3=df_maestra['Primer_Nombre'] + " " + df_maestra['Apellido_Paterno'] + " " + df_maestra['Apellido_Materno'],
    comb_4=df_maestra['Primer_Nombre'] + " " + df_maestra['Apellido_Materno'] + " " + df_maestra['Apellido_Paterno'],
    comb_5=df_maestra['Segundo_Nombre'] + " " + df_maestra['Primer_Nombre'] + " " + df_maestra['Apellido_Paterno'] + " " + df_maestra['Apellido_Materno'],
    comb_6=df_maestra['Segundo_Nombre'] + " " + df_maestra['Primer_Nombre'] + " " + df_maestra['Apellido_Materno'] + " " + df_maestra['Apellido_Paterno'],
    comb_7=df_maestra['Segundo_Nombre'] + " " + df_maestra['Apellido_Paterno'] + " " + df_maestra['Apellido_Materno'],
    comb_8=df_maestra['Segundo_Nombre'] + " " + df_maestra['Apellido_Materno'] + " " + df_maestra['Apellido_Paterno'],
    comb_9=df_maestra['Primer_Nombre'] + " " + df_maestra['Segundo_Nombre'] + " " + df_maestra['Apellido_Materno'],
    comb_10=df_maestra['Segundo_Nombre'] + " " + df_maestra['Primer_Nombre'] + " " + df_maestra['Apellido_Materno'],
    comb_11=df_maestra['Primer_Nombre'] + " " + df_maestra['Segundo_Nombre'] + " " + df_maestra['Apellido_Paterno'],
    comb_12=df_maestra['Segundo_Nombre'] + " " + df_maestra['Primer_Nombre'] + " " + df_maestra['Apellido_Paterno'],
    comb_13=df_maestra['Primer_Nombre'] + " " + df_maestra['Apellido_Paterno'],  # Primer Nombre + Apellido Paterno
    comb_14=df_maestra['Primer_Nombre'] + " " + df_maestra['Apellido_Materno'],  # Primer Nombre + Apellido Materno
    comb_15=df_maestra['Primer_Nombre'] + " " + df_maestra['Segundo_Nombre']    # Primer Nombre + Segundo Nombre
)

# Mostrar algunas combinaciones de ejemplo
print("Combinaciones generadas en la base maestra:")
print(df_maestra[['comb_1', 'comb_2', 'comb_3', 'comb_4', 'comb_5', 'comb_6', 'comb_7', 'comb_8', 'comb_9', 'comb_10', 'comb_11', 'comb_12', 'comb_13', 'comb_14', 'comb_15']].head())


Combinaciones generadas en la base maestra:
                            comb_1                           comb_2  \
0  ABELINO ENRIQUE JIMENEZ GAJARDO  ABELINO ENRIQUE GAJARDO JIMENEZ   
1     ABRAHAM BENJAMIN NOVOA LAGOS     ABRAHAM BENJAMIN LAGOS NOVOA   
2      ACCEL NICOLAS ABARCA PROUZA      ACCEL NICOLAS PROUZA ABARCA   
3          ADELAIDA BARROS BARROS           ADELAIDA BARROS  BARROS   
4       ADOLFO ANTONIO ROCCO ROCCO       ADOLFO ANTONIO ROCCO ROCCO   

                    comb_3                   comb_4  \
0  ABELINO JIMENEZ GAJARDO  ABELINO GAJARDO JIMENEZ   
1      ABRAHAM NOVOA LAGOS      ABRAHAM LAGOS NOVOA   
2      ACCEL ABARCA PROUZA      ACCEL PROUZA ABARCA   
3         ADELAIDA BARROS          ADELAIDA  BARROS   
4       ADOLFO ROCCO ROCCO       ADOLFO ROCCO ROCCO   

                            comb_5                           comb_6  \
0  ENRIQUE ABELINO JIMENEZ GAJARDO  ENRIQUE ABELINO GAJARDO JIMENEZ   
1     BENJAMIN ABRAHAM NOVOA LAGOS     BENJAMIN ABRAHAM 

In [6]:
# Filtrar registros de Scopus que tienen país 'Chile'
df_scopus_chile = df_scopus[df_scopus['pais_afiliacion'] == 'Chile']

# Verificar cuántos registros tienen Chile como país de afiliación
print(f"Total de registros de Scopus con país 'Chile': {len(df_scopus_chile)}")


Total de registros de Scopus con país 'Chile': 3011


In [10]:
# Inicializar un DataFrame vacío para almacenar los resultados del merge
df_merged_comb = pd.DataFrame()

# Realizar el merge con cada combinación de nombres de la base maestra y Scopus
for col in ['comb_1', 'comb_2', 'comb_3', 'comb_4', 'comb_5', 'comb_6', 'comb_7', 'comb_8', 'comb_9', 'comb_10', 'comb_11', 'comb_12', 'comb_13', 'comb_14', 'comb_15']:
    # Realizar el merge usando una combinación de nombres
    temp_merged = pd.merge(df_maestra, df_scopus_chile, how='left', left_on=col, right_on='nombre_completo_scopus')
    
    # Concatenar los resultados al DataFrame final
    df_merged_comb = pd.concat([df_merged_comb, temp_merged], ignore_index=True)

# Eliminar duplicados resultantes del proceso de concatenación
df_merged_comb.drop_duplicates(subset=['FOLIO', 'nombre_completo_scopus'], inplace=True)

# Verificar las primeras filas del DataFrame combinado
print("Primeras filas después del merge:")
print(df_merged_comb.head())


Primeras filas después del merge:
      FOLIO                           NOMBRE Primer_Nombre Segundo_Nombre  \
0  72160230  ABELINO ENRIQUE JIMENEZ GAJARDO       ABELINO        ENRIQUE   
1  21150571     ABRAHAM BENJAMIN NOVOA LAGOS       ABRAHAM       BENJAMIN   
2  72170568      ACCEL NICOLAS ABARCA PROUZA         ACCEL        NICOLAS   
3  72170089          ADELAIDA BARROS CAJDLER      ADELAIDA         BARROS   
4  72170561       ADOLFO ANTONIO ROCCO ROCCO        ADOLFO        ANTONIO   

  Apellido_Paterno Apellido_Materno SEXO_COMPLETO ESTADO BECA  \
0          JIMENEZ          GAJARDO     MASCULINO  NO VIGENTE   
1            NOVOA            LAGOS     MASCULINO  NO VIGENTE   
2           ABARCA           PROUZA     MASCULINO  NO VIGENTE   
3           BARROS                       FEMENINO  NO VIGENTE   
4            ROCCO            ROCCO     MASCULINO  NO VIGENTE   

   Ano de inicio beca    TIPO_BECA  ... cited_by_count citation_count  \
0                2015  BECAS CHILE  ...

In [11]:
# Contar cuántos nombres de la base de datos maestra fueron identificados en Scopus
nombres_identificados = df_merged_comb['nombre_completo_scopus'].notna().sum()

# Total de nombres en la base maestra considerando las 15 combinaciones
total_nombres_maestra = df_maestra[['comb_1', 'comb_2', 'comb_3', 'comb_4', 'comb_5', 
                                    'comb_6', 'comb_7', 'comb_8', 'comb_9', 'comb_10', 
                                    'comb_11', 'comb_12']].nunique().max()

# Mostrar resultados
print(f"Nombres identificados en Scopus: {nombres_identificados} de {total_nombres_maestra}")
print(f"Porcentaje de identificación: {nombres_identificados / total_nombres_maestra * 100:.2f}%")


Nombres identificados en Scopus: 543 de 3142
Porcentaje de identificación: 17.28%


In [12]:
# Guardar los resultados en un archivo CSV
output_csv_path = "C:/Users/Rodrigo/Desktop/Roxana Files/Scopus/resultados_maestra_scopus.csv"
df_merged_comb.to_csv(output_csv_path, index=False)

# Guardar los resultados en un archivo XLSX
output_xlsx_path = "C:/Users/Rodrigo/Desktop/Roxana Files/Scopus/resultados_maestra_scopus.xlsx"
df_merged_comb.to_excel(output_xlsx_path, index=False)

print(f"Resultados guardados en {output_csv_path} y {output_xlsx_path}")


Resultados guardados en C:/Users/Rodrigo/Desktop/Roxana Files/Scopus/resultados_maestra_scopus.csv y C:/Users/Rodrigo/Desktop/Roxana Files/Scopus/resultados_maestra_scopus.xlsx
